In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
%cd "/content/drive/MyDrive/data/3d data"

/content/drive/MyDrive/data/3d data


In [41]:
!pip install torchio
!pip install git+https://github.com/shijianjian/EfficientNet-PyTorch-3D

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shijianjian/EfficientNet-PyTorch-3D to /tmp/pip-req-build-2x_4q06a
  Running command git clone -q https://github.com/shijianjian/EfficientNet-PyTorch-3D /tmp/pip-req-build-2x_4q06a


In [42]:
import h5py # .h5 파일을 읽기 위한 패키지
import random
import pandas as pd
import numpy as np
import os
import glob

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import copy
import torchio as tio
from torchio.transforms import ZNormalization

from efficientnet_pytorch_3d import EfficientNet3D

from typing import Sequence, Callable
from tqdm.auto import tqdm

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 

In [43]:
CFG = {
    'BATCH_SIZE':128,
}

In [44]:
class CustomDataset(Dataset):
    def __init__(self, id_list, label_list, point_list, transforms= Sequence[Callable]):
        self.id_list = id_list
        self.label_list = label_list
        self.point_list = point_list
        self.transforms = transforms
    def __getitem__(self, index):
        image_id = self.id_list[index]
        
        # h5파일을 바로 접근하여 사용하면 학습 속도가 병목 현상으로 많이 느릴 수 있습니다.
        points = self.point_list[str(image_id)][:]
        image = self.get_vector(points)

        if self.label_list is not None:
            label = self.label_list[index]
            image=torch.Tensor(image).unsqueeze(0)
            if self.transforms is not None:
              image = self.transforms(image)
            return image, label
        else:
            image=torch.Tensor(image).unsqueeze(0)
            if self.transforms is not None:
              image = self.transforms(image)
            return image
    
    def get_vector(self, points, x_y_z=[16, 16, 16]):
        # 3D Points -> [16,16,16]
        xyzmin = np.min(points, axis=0) - 0.001
        xyzmax = np.max(points, axis=0) + 0.001

        diff = max(xyzmax-xyzmin) - (xyzmax-xyzmin)
        xyzmin = xyzmin - diff / 2
        xyzmax = xyzmax + diff / 2

        segments = []
        shape = []

        for i in range(3):
            # note the +1 in num 
            if type(x_y_z[i]) is not int:
                raise TypeError("x_y_z[{}] must be int".format(i))
            s, step = np.linspace(xyzmin[i], xyzmax[i], num=(x_y_z[i] + 1), retstep=True)
            segments.append(s)
            shape.append(step)

        n_voxels = x_y_z[0] * x_y_z[1] * x_y_z[2]
        n_x = x_y_z[0]
        n_y = x_y_z[1]
        n_z = x_y_z[2]

        structure = np.zeros((len(points), 4), dtype=int)
        structure[:,0] = np.searchsorted(segments[0], points[:,0]) - 1
        structure[:,1] = np.searchsorted(segments[1], points[:,1]) - 1
        structure[:,2] = np.searchsorted(segments[2], points[:,2]) - 1

        # i = ((y * n_x) + x) + (z * (n_x * n_y))
        structure[:,3] = ((structure[:,1] * n_x) + structure[:,0]) + (structure[:,2] * (n_x * n_y)) 

        vector = np.zeros(n_voxels)
        count = np.bincount(structure[:,3])
        vector[:len(count)] = count

        vector = vector.reshape(n_z, n_y, n_x)

        return vector

    def __len__(self):
        return len(self.id_list)

In [45]:
#override_params={'num_classes': 10},
class BaseModel(nn.Module):
    def __init__(self,cnn_model=None):
        super(BaseModel,self).__init__()
        if cnn_model=='eff3':
          self.model = EfficientNet3D.from_name("efficientnet-b3",override_params={'num_classes': 10},  in_channels=1)
        elif cnn_model=='eff0':
          self.model = EfficientNet3D.from_name("efficientnet-b0",override_params={'num_classes': 10},  in_channels=1)
        elif cnn_model=='eff4':
          self.model = EfficientNet3D.from_name("efficientnet-b4",override_params={'num_classes': 10},  in_channels=1)

    def forward(self,x):

        x = self.model(x)

        return x

In [46]:
#override_params={'num_classes': 10},
class MoreModel(nn.Module):
    def __init__(self,cnn_model=None):
        super(MoreModel,self).__init__()
        if cnn_model=='eff3':
          self.model = EfficientNet3D.from_name("efficientnet-b3",  in_channels=1)
        elif cnn_model=='eff0':
          self.model = EfficientNet3D.from_name("efficientnet-b0",  in_channels=1)
        elif cnn_model=='eff4':
          self.model = EfficientNet3D.from_name("efficientnet-b4",  in_channels=1)
        self.SiLU=nn.SiLU(inplace=False)
        self.dropout=nn.Dropout(p=0.2)
        self.classifier = nn.Linear(1000, 10)
        nn.init.xavier_normal_(self.classifier.weight)

    def forward(self,x):

        x = self.model(x)
        x = self.SiLU(x)
        x = self.dropout(x)
        x = self.classifier(x)
        return x

In [47]:
test_df = pd.read_csv('./sample_submission.csv')
test_points = h5py.File('./test.h5', 'r')

In [48]:
test_transform = tio.Compose([

    tio.Resize((128,128,128)),
    ZNormalization(),
])

In [49]:
test_dataset = CustomDataset(test_df['ID'].values, None, test_points,transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

In [50]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [51]:
models={"eff4 solution 72 LR Ap degree 70":"eff4",
}



In [52]:
best_models = []
for k, v in models.items():
  model=MoreModel(cnn_model=v)
  model.load_state_dict(torch.load('/content/drive/MyDrive/data/3d data/models/{}.pt'.format(k), map_location=device), strict=False)
  best_models.append(copy.deepcopy(model))


In [53]:
def predict(test_loader, device):

    model_preds = []
    with torch.no_grad():
        for data in tqdm(iter(test_loader)):
          batch_pred=0
          data = data.float().to(device)
          with torch.cuda.amp.autocast():
            for model in best_models:
              model.to(device)
              model.eval()
              batch_pred += model(data)
            model_preds += batch_pred.argmax(1).detach().cpu().numpy().tolist()
    
    return model_preds

In [ ]:
preds = predict(test_loader, device)

In [ ]:
test_df['label'] = preds
test_df.to_csv('./submit.csv', index=False)